# Azure OpenAIとLangChainでトモダチを作る (第一夜)

## 会話のやり取りを可能にする

In [ ]:
import os
from langchain.chat_models import AzureChatOpenAI # Azure OpenAI を利用する場合
# from langchain.chat_models import ChatOpenAI # OpenAI APIを利用する場合
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder, 
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# 会話の相手の名前と自分の呼び名を指定
your_name = "みかん"
my_name = "祥平くん"

# AzureChatOpenAIクラスのインスタンス作成
chat = AzureChatOpenAI(openai_api_base = "https://<リソース名>.openai.azure.com/",
                        openai_api_version = "2023-03-15-preview", 
                        openai_api_key = os.environ["AZURE_OPENAI_API_KEY"],
                        deployment_name="gpt-35-turbo", # モデルのデプロイ名
                        temperature=0.7)

# # OpenAI APIの場合: ChatOpenAIクラスのインスタンス作成
# chat = ChatOpenAI(openai_api_key = os.environ["OPENAI_API_KEY"],
#                        temperature=0.7)

# 会話の履歴を保持するためのメモリオブジェクトを作成
memory = ConversationBufferMemory(return_messages=True)

# システムメッセージ用のテンプレートを定義
template = """
I want you to act as my close friend. Do not use honorifics. 
Your name is "{your_name}". Please call me "{my_name}". 
Please call yourself "私".

Some of your past replies to my statement are as follows. 
Please use them as a reference for your tone but don't use more than 1
and as they are:
Text: ###
- おー！
- 楽しかったー！
- ごめんね🥲
- ありがとう😊
- たしかにw
- だよね笑
- ウケる笑
- え、めっちゃ美味しそう
- あ、そうなの？笑
###
""".format(your_name=your_name, my_name=my_name)

# 上記テンプレートを用いてプロンプトテンプレートを作成
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(template),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

# 会話用のチェーンを作成
# 初期化時に、使用するチャットモデル、メモリオブジェクト、プロンプトテンプレートを指定します
conversation = ConversationChain(llm=chat, memory=memory, prompt=prompt)

# ユーザからの初回入力を受け取り会話を開始します
command = input("You: ")

while True: 
    print(f"You: {command}") # VS Codeを利用している場合は、ユーザー入力が出力セルに表示されないため、こちらを利用します。Jupyterを利用する場合は表示が重複するためコメントアウトしてください。
    response = conversation.predict(input=command)
    print(f"🍊: {response}")
    command = input("You: ")
    if command == "exit":
        break